In [1]:
# SPDX-License-Identifier: GPL-3.0-or-later

## In this notebook we test the effects of some assumptions in the electrostatic forward model

In [1]:
%matplotlib widget

In [3]:
import numpy as np
import pandas as pd
import os
from scipy.stats import linregress

from matplotlib import pyplot as plt;
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 6
plt.rcParams["lines.linewidth"] = 0.5
plt.rcParams["axes.linewidth"] = 0.5
plt.rcParams["axes.spines.top"] = False
plt.rcParams["xtick.major.width"] = 0.5 
plt.rcParams["xtick.minor.width"] = 0.5 
plt.rcParams["ytick.major.width"] = 0.5 
plt.rcParams["ytick.minor.width"] = 0.5
plt.rcParams["xtick.major.size"] = 3.5 * 1.1
plt.rcParams["xtick.minor.size"] = 2 * 1.1
plt.rcParams["ytick.major.size"] = 3.5 * 1.1
plt.rcParams["ytick.minor.size"] = 2 * 1.1


import conntility


In [4]:
savefig_cfg = {"transparent":True, "dpi": 300}
legend_cfg = {"frameon": False, "handletextpad": 0.1}
tight_layout_cfg = {"pad": 0.5}

## First, we check the number of somas that are within 10 $\mu$m of a neuropixels contact
### This shows that the assumption that somas are infinitesimal points have a limited impact on the results

In [5]:
import bluepysnap as bp
path_to_simulation = '/home/joseph-tharayil/Documents/bluebrainStuff/csd_paper/electrodes/'
s = bp.Simulation(path_to_simulation+'simulation_config.json') # Loads simulation object
population_name = 'S1nonbarrel_neurons'


In [6]:
c = s.circuit

In [7]:
hex0Ids = c.nodes.ids('hex0').get_ids()

In [8]:
n = c.nodes['S1nonbarrel_neurons']

In [9]:
somaLocs = n.get(hex0Ids,properties=['x','y','z'])

In [10]:
neuropixelsLocs = pd.read_csv('~/Downloads/electrode_csv.csv').iloc[:,1:4]

In [11]:
distances = []
for s in np.array(somaLocs):
    distances.append(np.linalg.norm(s-neuropixelsLocs,axis=1))
distances = np.array(distances)

In [12]:
np.sum(np.any(distances<10,axis=1))

87

### Next, we compare the weights obtained with infinitesimal electrodes with those calculated with a minimum 6 $\mu$m distance between the neural segment and the electrode

In [13]:
import h5py

In [14]:
# Loads the weights file
coeffsFile = 'lfp_fullNeuropixels.h5'

coeffs = h5py.File(coeffsFile)

coeffsFileC = 'lfp_fullNeuropixels_finite.h5'

coeffs_finite = h5py.File(coeffsFileC)


In [15]:
allValues = coeffs['electrodes']['S1nonbarrel_neurons']['scaling_factors'][:][:,:-1]
allValuesFinite = coeffs_finite['electrodes']['S1nonbarrel_neurons']['scaling_factors'][:][:,:-1]


In [16]:
diffs = allValues - allValuesFinite

In [2]:
segmentsWithinSphere = (diffs!=0).astype(int) # Segments within a spherical electrode will have a different coefficient under the assumption that electrodes are infinitesimal


NameError: name 'diffs' is not defined

In [18]:
segmentsOutsideSphere = np.ones_like(segmentsWithinSphere)-segmentsWithinSphere # In contrast, segments outside of the spheres will have the same coefficient under the assumption that electrodes are infinitesimal 


In [19]:
diffIdx = np.where(diffs != 0)

In [20]:
# In this cell, we find the IDs of neurons which have at least one segment within a spherical electrode
gidsWithinSpheres = []
offsets = coeffs['S1nonbarrel_neurons']['offsets'][:]
for idx in diffIdx[0]:
    gid = np.where(idx>offsets)[0][-1]
    gidsWithinSpheres.append(gid)

In [32]:
# Here, we iterate through the segments within spheres, and find the segment with the smallest distance to the electrode that is part of a cell that spikes within the first two seconds of the simulation

spikes = np.genfromtxt('out.dat',skip_header=1)
spikingGids = spikes[:,1]

offsets = coeffs['S1nonbarrel_neurons']['offsets'][:]
sortedDiffs = np.sort(np.abs(diffs.flatten()))
for i in np.arange(1,1000):
    segIdx = np.where(np.abs(diffs)==sortedDiffs[-i])[0]
    offsetIdx = np.min(np.where(offsets>segIdx))
    gid = coeffs['S1nonbarrel_neurons']['node_ids'][offsetIdx]
    if gid in spikingGids:
        print(gid)
        print(np.where(np.abs(diffs)==sortedDiffs[-i])[1])
        break

180725
[28]


In [21]:
np.sort(np.abs(diffs.flatten()))[-i] # This is the difference in the coefficient between the finite and infinitesimal electrode, for the selected gid

4.222574924772946e-05

In [22]:
coeffIdx = np.argsort(np.abs(diffs.flatten()))[-209] #

407140066

In [23]:
coeffInfinitesimal = allValues.flatten()[coeffIdx]

9.01062977109952e-05

In [24]:
distance = 1/(coeffInfinitesimal * 4 *np.pi * 0.277)*1e-3 # Distance between the segment and the electrode, in um

3188.2709430701507

In [38]:
minDistanceInModel = 1/(4*np.pi*.277*np.max(allValues[diffIdx])) * 1e-3
print(minDistanceInModel) # Minimum segment-electrode distance, in um

0.6867981119867946


In [28]:
distanceRatio = distance / minDistanceInModel

4.11764705882353